In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun Mar 10 16:14:41 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   60C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 54.8 gigabytes of available RAM

You are using a high-RAM runtime!


In [3]:
# First, run this installation command in your Colab notebook to ensure all necessary libraries are installed.
!pip install torch transformers[torch] datasets accelerate -U

In [4]:
import os

os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

In [5]:
# Install necessary libraries
!pip install torch transformers[torch] datasets accelerate -U

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
# Importing necessary libraries
from transformers import GPTNeoForCausalLM, GPT2Tokenizer, Trainer, TrainingArguments, GPTNeoConfig
from datasets import load_dataset, DatasetDict
from transformers import DataCollatorForLanguageModeling

import os
import torch
from transformers import GPTNeoForCausalLM, GPT2Tokenizer, Trainer, TrainingArguments

# Set PYTORCH_CUDA_ALLOC_CONF to avoid fragmentation
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

# Initialize the tokenizer
#model_name = 'EleutherAI/gpt-neo-1.3B'
model_name = 'EleutherAI/gpt-neo-125m'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Explicitly set the tokenizer's padding token
tokenizer.pad_token = tokenizer.eos_token

# Initialize the model with gradient checkpointing enabled for memory efficiency
model_config = GPTNeoConfig.from_pretrained(model_name, gradient_checkpointing=True)
model = GPTNeoForCausalLM.from_pretrained(model_name, config=model_config)

# Function to tokenize the dataset correctly, considering batched inputs
def tokenize_function(examples):
    concatenated_messages = [" ".join([message['content'] for message in example_messages]) for example_messages in examples['messages']]
    return tokenizer(concatenated_messages, truncation=True, padding="max_length", max_length=128)

# Paths to your training and testing dataset
train_path = '/content/drive/My Drive/Colab_Notebooks/datasets/train_data.jsonl'
test_path = '/content/drive/My Drive/Colab_Notebooks/datasets/test_data.jsonl'

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [7]:
# Load and preprocess the dataset
raw_datasets = DatasetDict({
    'train': load_dataset('json', data_files=train_path, split='train'),
    'test': load_dataset('json', data_files=test_path, split='train')
})

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True, remove_columns=['messages'])

# Data collator to dynamically pad the inputs and labels
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Define the training arguments with reduced batch size and enabled mixed precision
training_args = TrainingArguments(
    output_dir='./results',
    overwrite_output_dir=True,
    num_train_epochs=2,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    logging_dir='./logs',
    logging_steps=100,
    save_strategy="epoch",
    fp16=True  # Enable mixed precision training
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
    data_collator=data_collator,
)

# Start training
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Step,Training Loss


Checkpoint destination directory ./results/checkpoint-2 already exists and is non-empty. Saving will proceed but saved results may be invalid.


TrainOutput(global_step=2, training_loss=4.233532428741455, metrics={'train_runtime': 11.6745, 'train_samples_per_second': 1.884, 'train_steps_per_second': 0.171, 'total_flos': 979526615040.0, 'train_loss': 4.233532428741455, 'epoch': 1.33})

In [8]:
# Save the fine-tuned model and tokenizer for future use
model.save_pretrained('/content/drive/My Drive/Colab_Notebooks/fine_tuned_model_gpt_neo')
tokenizer.save_pretrained('/content/drive/My Drive/Colab_Notebooks/fine_tuned_tokenizer_gpt_neo')

('/content/drive/My Drive/Colab_Notebooks/fine_tuned_tokenizer_gpt_neo/tokenizer_config.json',
 '/content/drive/My Drive/Colab_Notebooks/fine_tuned_tokenizer_gpt_neo/special_tokens_map.json',
 '/content/drive/My Drive/Colab_Notebooks/fine_tuned_tokenizer_gpt_neo/vocab.json',
 '/content/drive/My Drive/Colab_Notebooks/fine_tuned_tokenizer_gpt_neo/merges.txt',
 '/content/drive/My Drive/Colab_Notebooks/fine_tuned_tokenizer_gpt_neo/added_tokens.json')

In [16]:
# Load the fine-tuned model and tokenizer for inference
model = GPTNeoForCausalLM.from_pretrained('/content/drive/My Drive/Colab_Notebooks/fine_tuned_model_gpt_neo')
tokenizer = GPT2Tokenizer.from_pretrained('/content/drive/My Drive/Colab_Notebooks/fine_tuned_tokenizer_gpt_neo')

# Example of user query for inference
user_query = "Tell me something interesting about space."

# Encode the user query to tensor of input IDs
input_ids = tokenizer.encode(user_query, return_tensors='pt')

# Generate a sequence of tokens in response to the input query
# output = model.generate(input_ids, max_length=100, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)
output = model.generate(
    input_ids,
    max_length=300,
    num_return_sequences=1,
    pad_token_id=tokenizer.eos_token_id,
    temperature=0.8,  # Lower temperature
    top_p=0.92,  # Nucleus sampling
    top_k=50,  # Top-k sampling
    repetition_penalty=1.2,  # Penalize repetition
    no_repeat_ngram_size=2  # Prevent repeating n-grams
)

# Decode the generated sequence to a human-readable string
# response = tokenizer.decode(output[0], skip_special_tokens=True)
response = tokenizer.decode(
    model.generate(
        input_ids,
        max_length=50,
        num_return_sequences=1,
        pad_token_id=tokenizer.eos_token_id,
        temperature=0.8,  # Adjust temperature
        top_k=50,  # Top-k sampling
        top_p=0.92,  # Nucleus (top-p) sampling
        repetition_penalty=1.2,  # Apply repetition penalty
        no_repeat_ngram_size=2,  # Prevent repeating n-grams
        do_sample=True  # Enable sampling to use temperature, top_k, and top_p
    )[0],
    skip_special_tokens=True
)

print("Generated response:", response)

Generated response: Tell me something interesting about space.

I love the way the planets and stars are portrayed in the movies. But, there’s a huge problem with the depiction of the universe in those films. For a long time, they were depicted
